# Day 8: Haunted Wasteland

## Part One

You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about ghosts a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:

In [22]:
example1 = [
    "RL",
    "AAA = (BBB, CCC)",
    "BBB = (DDD, EEE)",
    "CCC = (ZZZ, GGG)",
    "DDD = (DDD, DDD)",
    "EEE = (EEE, EEE)",
    "GGG = (GGG, GGG)",
    "ZZZ = (ZZZ, ZZZ)"
]

8-element Vector{String}:
 "RL"
 "AAA = (BBB, CCC)"
 "BBB = (DDD, EEE)"
 "CCC = (ZZZ, GGG)"
 "DDD = (DDD, DDD)"
 "EEE = (EEE, EEE)"
 "GGG = (GGG, GGG)"
 "ZZZ = (ZZZ, ZZZ)"

Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:

In [23]:
example2 = [
    "LLR",
    "AAA = (BBB, BBB)",
    "BBB = (AAA, ZZZ)",
    "ZZZ = (ZZZ, ZZZ)"
]

4-element Vector{String}:
 "LLR"
 "AAA = (BBB, BBB)"
 "BBB = (AAA, ZZZ)"
 "ZZZ = (ZZZ, ZZZ)"

Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?

In [24]:
using ResumableFunctions

struct Node
    key::AbstractString
    left::AbstractString
    right::AbstractString
end

function parsegraph(contents)
    Dict(
        map(
            function (content)
                (key, left, right) = match(r"(\w{3}) = \((\w{3}), (\w{3})\)", content)
                "$key" => Node(key, left, right)
            end,
            contents
        )
    )
end

@resumable function walk(start, graph, steps)
    stepindex = firstindex(steps)
    node = start
    while true
        s = steps[stepindex]
        stepindex += 1
        if stepindex > lastindex(steps)
            stepindex = firstindex(steps)
        end
        if s === 'L'
            node = get(graph, node.left, Node(node.left, node.left, node.left))
        else
            node = get(graph, node.right, Node(node.right, node.right, node.right))
        end
        @yield node
    end
end

walk (generic function with 1 method)

In [25]:
function partone(contents)
   graph = parsegraph(contents[begin+1:end])
   for (node, step) in zip(walk(graph["AAA"], graph, contents[begin]), Iterators.countfrom(1))
      if node.key == "ZZZ"
         return step
      end
   end
end

println(partone(example1))
println(partone(example2))


2
6


## Part Two

The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take significantly more steps to escape!

What if the map isn't for people - what if the map is for ghosts? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

For example:

In [26]:
example3 = [
    "LR",
    "11A = (11B, XXX)",
    "11B = (XXX, 11Z)",
    "11Z = (11B, XXX)",
    "22A = (22B, XXX)",
    "22B = (22C, 22C)",
    "22C = (22Z, 22Z)",
    "22Z = (22B, 22B)",
    "XXX = (XXX, XXX)"
]

9-element Vector{String}:
 "LR"
 "11A = (11B, XXX)"
 "11B = (XXX, 11Z)"
 "11Z = (11B, XXX)"
 "22A = (22B, XXX)"
 "22B = (22C, 22C)"
 "22C = (22Z, 22Z)"
 "22Z = (22B, 22B)"
 "XXX = (XXX, XXX)"

Here, there are two starting nodes, 11A and 22A (because they both end with A). As you follow each left/right instruction, use that instruction to simultaneously navigate away from both nodes you're currently on. Repeat this process until all of the nodes you're currently on end with Z. (If only some of the nodes you're on end with Z, they act like any other node and you continue as normal.) In this example, you would proceed as follows:

- Step 0: You are at 11A and 22A.
- Step 1: You choose all of the left paths, leading you to 11B and 22B.
- Step 2: You choose all of the right paths, leading you to 11Z and 22C.
- Step 3: You choose all of the left paths, leading you to 11B and 22Z.
- Step 4: You choose all of the right paths, leading you to 11Z and 22B.
- Step 5: You choose all of the left paths, leading you to 11B and 22C.
- Step 6: You choose all of the right paths, leading you to 11Z and 22Z.

So, in this example, you end up entirely on nodes that end in Z after 6 steps.

Simultaneously start on every node that ends with A. How many steps does it take before you're only on nodes that end with Z?

In [27]:
using Primes

function findnodes(graph)
    [n for n = values(graph) if n.key[end] == 'A']
end

function findperiod(node, graph, steps)
    tracking = Dict()
    for (step, curnode) in zip(Iterators.countfrom(1), walk(node, graph, steps))
        if curnode.key[end] == 'Z'
            existing = get!(tracking, curnode.key, step)
            if existing != step
                return existing
            end
        end
    end
end

function parttwo(contents)
    graph = parsegraph(contents[begin+1:end])
    nodes = findnodes(graph)
    steps = contents[begin]
    prod(map(v -> v[1], union(map(n -> Primes.factor(findperiod(n, graph, steps)), nodes)...)))
end

parttwo(example3)

6

## Results

In [28]:
include("./aoc.jl")
execute(8, partone, parttwo)

18113
12315788159977
